# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [30]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [31]:
weather_csv_file = "../WeatherPy/city_data_output.csv"

weather_df = pd.read_csv(weather_csv_file)

weather_df.head()


,Unnamed: 0,City,Temperature(C),Humidity(%),Cloudiness(%),Wind Speed(kph),Latitude,Longitude
0,0,punta arenas,4.06,81.0,0.0,6.17,-53.1500,-70.9167
1,2,olafsvik,9.15,76.0,100.0,6.96,64.8945,-23.7142
2,3,richards bay,19.37,59.0,100.0,6.80,-28.7830,32.0377
3,4,yellowknife,17.29,63.0,20.0,2.06,62.4560,-114.3525
4,5,mareeba,23.57,77.0,52.0,2.89,-17.0000,145.4333


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [32]:
# Handle NaN values
weather_df = weather_df.dropna()

# Store latitude and longitude in locations
locations = weather_df[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
rating = weather_df["Humidity(%)"].astype(float)

In [33]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=max(rating),
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [34]:
# Narrow down the DataFrame to find your ideal weather condition.
filtered_weather_df = weather_df

# A max temperature lower than 26 degrees but higher than 20.
filtered_weather_df = filtered_weather_df.loc[(filtered_weather_df["Temperature(C)"] < 26) & (filtered_weather_df["Temperature(C)"] > 20)]

# Wind speed less than 16 kph.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Wind Speed(kph)"] < 16]

# Zero cloudiness.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Cloudiness(%)"] == 0]

# Drop any rows with null values
filtered_weather_df = filtered_weather_df.dropna()

filtered_weather_df

,Unnamed: 0,City,Temperature(C),Humidity(%),Cloudiness(%),Wind Speed(kph),Latitude,Longitude
53,59,zanjan,23.83,20.0,0.0,2.06,36.6736,48.4787
149,157,cherskiy,20.97,22.0,0.0,4.44,68.7500,161.3000
179,188,mpika,20.72,43.0,0.0,1.88,-11.8343,31.4529
205,217,lahaina,25.45,83.0,0.0,5.20,20.8783,-156.6825
232,246,mwinilunga,20.21,39.0,0.0,1.56,-11.7358,24.4293
241,255,goderich,21.18,46.0,0.0,4.12,43.7501,-81.7165
250,266,surt,24.26,63.0,0.0,0.87,31.2089,16.5887
251,267,nashville,25.03,85.0,0.0,0.00,36.1659,-86.7844
304,324,ankazoabo,24.87,43.0,0.0,2.69,-22.2833,44.5167
306,326,ajdabiya,25.23,57.0,0.0,1.64,30.7554,20.2263


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [39]:
hotel_df = filtered_weather_df

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    # add keyword to params dict
    params['Location'] = f"{lat},{lng}"
    
    
    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']

    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 53: zanjan.
Missing field/result... skipping.
------------
Retrieving Results for Index 149: cherskiy.
Missing field/result... skipping.
------------
Retrieving Results for Index 179: mpika.
Missing field/result... skipping.
------------
Retrieving Results for Index 205: lahaina.
Missing field/result... skipping.
------------
Retrieving Results for Index 232: mwinilunga.
Missing field/result... skipping.
------------
Retrieving Results for Index 241: goderich.
Missing field/result... skipping.
------------
Retrieving Results for Index 250: surt.
Missing field/result... skipping.
------------
Retrieving Results for Index 251: nashville.
Missing field/result... skipping.
------------
Retrieving Results for Index 304: ankazoabo.
Missing field/result... skipping.
------------
Retrieving Results for Index 306: ajdabiya.
Missing field/result... skipping.
------------
Retrieving Results for Index 346: marsa matruh.
Missing field/result... skipping.
------------
Re

In [36]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'Hotel Name'

In [ ]:
# Add marker layer ontop of heat map


# Display figure
